# Przydatne techniki

W tym laboratorium główny nacisk zostanie położony na obsługę wejścia/wyjścia, obsługa popularnych formatów w języku Python (_Excel_/_JSON_/_Parquet_) oraz operacje związane przeglądanie plików i katalogów w systemie plików. Język Python jest niezależny od systemu operacyjnego, który definiuje w jaki sposób fizycznie obsługiwane są pliki. Wiele wysiłku włożono w utworzenie warstwy abstrakcji, która pośredniczy w wywoływaniu funkcji systemowych. Z tego też powodu łączenie katalogu i kolejnych podkatalogów wymaga użycia odpowiedniej funkcji, nie zwykłej konkatenacji łańcuchów znaków zawierających kolejne katalogi. Dodatkowo zostaną przedstawione tematy pokrewne, jak obsługa plikowej bazy danych `sqlite` (transakcyjnej i spełniającej zasady _ACID_) przy użyciu _ORM_ (Object-relational mapping), do czego zostanie użyty moduł `SqlAlchemy` oraz serializacja, deserializacja obiektów przy użyciu modułu `pickle`.

## System plików

Pierwszym przykładem użycia Pythona do obsługi systemu plików będzie rekurencyjne wyświetlenie listy plików. Do tego celu można użyć bardzo przydatnej funkcji `os.walk`, która wyświetla wszystkie pliki i podkatalogi w katalogu podanym w parametrze. Wszystkie nazwy przekazane z funkcji nie zawierają pełnej ścieżki, a jedynie jej relatywną część. W wielu przypadkach potrzebna jest absolutna ścieżka do bieżącego katalogu. Do jej pobrania można użyć wbudowanej funkcji `os.getcwd()` (akronim od _get current working directory_).

In [18]:
from os import getcwd, walk
from os.path import join

def get_files(folder):
    for folderName, sub_folders, files_names in walk(folder):
        for file_name in files_names:
            print(join(folderName, file_name))
        for sub_folder in sub_folders:
            get_files(join(folder, sub_folder))

get_files(getcwd())

C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\collections.ipynb
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\functions.ipynb
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\introduction.ipynb
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\objective_programming.ipynb
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\usefull_techiques.ipynb
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\.idea\.gitignore
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\.idea\misc.xml
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki programowania obiektowego II\.idea\modules.xml
C:\Users\lukaszstrak\Dropbox\Publikacje\Dydaktyka\Nowoczesne języki p

W linii 6 została użyta funkcja `join` modułu `os.path`. Służy do konkatenacji wielu folderów, które składają się na pełną ścieżkę.

Poniższa lista zawiera zestaw funkcji przydatnych do obsługi plików i katalogów na poziomie systemy plików.

| Nazwa funkcji | Opis | Przykład użycia |
|-|-|-|
| `shutil.copy` | Funkcja umożliwia kopiowanie pojedynczych plików. | `shutil.copy(full_path_src, full_path_dest)` |
| `shutil.copytree` | Funkcja umożliwia kopiowanie całych katalogów wraz z zawartością. | `shutil.copytree(path_src, path_dest)` |
| `shutil.move` | Funkcja umożliwia zmianę nazwy pliku oraz przenoszenie pliku do innego katalogu. | `shutil.move(path_src, path_dest)` |
| `shutil.rmtree` | Funkcja umożliwia kasowanie katalogu wraz z zawartością. | `shutil.rmtree(path)` |
| `os.unlink` | Funkcja umożliwia kasowanie pliku. | `shutil.unlink(path)` |
| `os.path.splitext` | Zwraca tuple na, którą składa się nazwa pliku oraz jego rozszerzenie. W przypadku, gdy podany plik zawiera pełną ścieżkę, zostanie ona zwrócona jako część nazwy pliku. | `file_name, ext = os.path.splitext(file_name)` |


Tworzenie bezpiecznych plików tymczasowych jest bardzo przydatne szczególnie w testach jednostkowych. W języku Python dostępny jest moduł `tempfile`, który znacznie ułatwia tworzenie tego typu plików. Funkcje `mkdtemp` i `mktemp` służą kolejno do tworzenia tymczasowego katalogu i pliku.

In [19]:
from tempfile import mkdtemp
from shutil import rmtree
from os.path import join
temp_dir = mkdtemp()
try:
    print(f'Saving file to: {temp_dir}')
    with open(join(temp_dir, 'file.txt'), 'w+') as f:
        f.write('Test')
        f.flush()
finally:
    rmtree(temp_dir)

print('Success')

Saving file to: C:\Users\LUKASZ~1\AppData\Local\Temp\tmp0pmza_f1
Success


W powyższym kodzie to programista jest odpowiedzialny za zwalnianie pliku/katalogu tymczasowego. Moduł `tempfile` umożliwia tworzenie plików, które w momencie opuszczenia bloku kodu, zostaną zwolnione automatycznie.

In [21]:
from tempfile import NamedTemporaryFile
from os.path import exists

file_name = None
with NamedTemporaryFile('w+') as f:
    f.write('test')
    file_name = f.name
    print(f'Fle {file_name} exists: {exists(file_name)}')

print(f'Fle {file_name} exists: {exists(file_name)}')

Fle C:\Users\LUKASZ~1\AppData\Local\Temp\tmp82e50cic exists: True
Fle C:\Users\LUKASZ~1\AppData\Local\Temp\tmp82e50cic exists: False


Dość istotny jest tutaj fakt, że gdy plik zostanie zamknięty funkcją `f.close()`, zostanie skasowany. W przypadku katalogu można użyć klasy `tempfile.TemporaryDirectory`, której schemat użycia jest taki sam.

## Obsługa formatów plików

### Format `Xlsx`

Moduł `openpyxl` pozwala na obsługę formatu plików `xlsx`. Z tego samego modułu korzysta również bardzo popularny moduł `pandas`, który umożliwia unifikację przetwarzania danych bez względu na format źródłowy. Moduł `openpyxl` nie znajduje się na liście modułów standardowych i należy go doinstalować za pomocą polecenia `pip install openpyxl`.

In [42]:
from openpyxl import Workbook, load_workbook
from typing import Dict
from tempfile import mktemp
from itertools import product
from os import unlink

def describe(file_name, sheet_name):
    wb = load_workbook(file_name)
    print(f'Sheets: {wb.sheetnames}')
    print(f'Filled rows: {wb[sheet_name].max_row}')
    print(f'Filled columns: {wb[sheet_name].max_column}')
    for cell in product(['A', 'B'], ['1', '2', '3']):
        cell_s = "".join(cell)
        print(f'Cell: {cell_s}: {wb[sheet_name][cell_s].value}')
    # można też użyć
    # for coll_obj in list(wb[sheet_name].columns)[0]:
    #     print(f'Cell: {coll_obj.value}')
    # lub
    # wb[sheet_name].cell(row=1, column=1).value

def create(data: Dict[str, int], file_name, sheet_name):
    wb = Workbook()
    sheet = wb.create_sheet(sheet_name)
    for cell_key, cell_value in data.items():
        sheet[cell_key] = cell_value
    wb.save(file_name)

file_name = mktemp(suffix='.xlsx')

print(f'Using file: {file_name}')

try:
    data = { 'A1': 10, 'A2': 20, 'A3': 30, 'B1': 11, 'B2': 12, 'B3': 13 }
    create(data, file_name, 'test')
    describe(file_name, 'test')

finally:
    unlink(file_name)

Using file: C:\Users\LUKASZ~1\AppData\Local\Temp\tmpuz_wc7rt.xlsx
Sheets: ['Sheet', 'test']
Filled rows: 3
Filled columns: 2
Cell: A1: 10
Cell: A2: 20
Cell: A3: 30
Cell: B1: 11
Cell: B2: 12
Cell: B3: 13


Przypisując odpowiedniej komórce formułę w postaci łańcucha znaków tj. `sheet['A4'] = '=SUM(A1:A3)'` tworzymy formułę wyliczaną automatycznie. W module `openpyxl.utils` można znaleźć między innymi dwie funkcje konwertujące liczby na litery i odwrotnie, co ułatwia iterowanie między komórkami.

In [36]:
from openpyxl.utils import get_column_letter, column_index_from_string

print('Letters:')
print(",".join([get_column_letter(i) for i in range(1, 100)]))
print(column_index_from_string('A'))
print(column_index_from_string('AB'))
print(column_index_from_string('ABC'))

Letters:
A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,AA,AB,AC,AD,AE,AF,AG,AH,AI,AJ,AK,AL,AM,AN,AO,AP,AQ,AR,AS,AT,AU,AV,AW,AX,AY,AZ,BA,BB,BC,BD,BE,BF,BG,BH,BI,BJ,BK,BL,BM,BN,BO,BP,BQ,BR,BS,BT,BU,BV,BW,BX,BY,BZ,CA,CB,CC,CD,CE,CF,CG,CH,CI,CJ,CK,CL,CM,CN,CO,CP,CQ,CR,CS,CT,CU
1
28
731


### Format _JSON_

Wczytywanie i zapisywanie formatu _JSON_ polega głównie na operowaniu na słownikach. Funkcja `loads` zwraca słownik na podstawie przekazanego łańcucha znaków w formacie _JSON_. Z kolei funkcja `dumps` zapisuje słownik w tym formacie. Można również użyć funkcji `load` i `dump`, która oczekuje strumienia do pliku zapisanego w formacie _JSON_ (otwarcie strumienia następuje w funkcji `open`).

In [43]:
from json import loads, dumps

json_dict = loads('{ "file_name": "test", "is_zipped": false }')
print(f'File name: {json_dict["file_name"]}')
print(f'Is zipped: {json_dict["is_zipped"]}')
print(f'Dumped: {dumps(json_dict)}')

File name: test
Is zipped: False
Dumped: {"file_name": "test", "is_zipped": false}


Warto w tym miejscu należy zwrócić uwagę na konwersję typów. Wartość typu `bool` w formacie _JSON_ została skonwertowana na odpowiednik w języku Python.

### Format _Parquet_

Format `Parquet` jest używany m. in. w _Apache Spark_ (_PySpark_) czy module `pandas` i jest jednym z podstawowych formatów w ekosystemie firmy _Apache_. Pod względem struktury jest to format binarny, kolumnowy z silnym typowaniem danych. W bibliotece standardowej nie ma obsługi tego formatu, jednak znajduje się wiele implementacji w `pip`. Przykłady utylizują bibliotekę `fastparquet`, a do tworzenia bibliotekę `pandas`.

In [70]:
import pandas as pd
from tempfile import mktemp
from os import unlink
from fastparquet import ParquetFile

data = { 'A': list(range(1,5)), 'B': list(range(6,10)) }

print(f'Using data: {data}')

df = pd.DataFrame.from_dict(data)

temp_file = mktemp()

try:
    df.to_parquet(temp_file, compression='GZIP')
    pf = ParquetFile(temp_file)
    print(f'Readed columns: {pf.columns}')
    print(f'File info: {pf.info}')
    df = pf.to_pandas()
    for value_a, value_b in df.itertuples(index=False):
        print(f'Values: {value_a}, {value_b}')

finally:
    unlink(temp_file)

Using data: {'A': [1, 2, 3, 4], 'B': [6, 7, 8, 9]}
Readed columns: ['A', 'B']
File info: {'name': 'C:/Users/LUKASZ~1/AppData/Local/Temp/tmpkwxu34tp', 'columns': ['A', 'B'], 'partitions': [], 'rows': 4}
Values: 1, 6
Values: 2, 7
Values: 3, 8
Values: 4, 9


Format `Parquet` jest szczególnie przydatny, gdy przetwarzamy duże zbiory danych i wynik należy zapisać, gdyż kolejny krok przetwarzania może zmodyfikować aktualnie przeprocesowane dane (_check point_).

## Baza danych _sqlite_ i podstawy `SqlAlchemy`

Mapowanie bazy danych na klasy jest bardzo przydatne w większych projektach. Głównie za sprawą podpowiadania składni oraz teoretycznej niezależności działania programu od warstwy przechowywania danych (baza danych). Biblioteka `SqlAlchemy` stanowi warstwę pośrednią miedzy fizycznymi operacjami na konkretnej bazie danych oraz logicznych operacji jak odczytywanie czy modyfikowanie danych. Co więcej, model danych to klasa w języku Python, co pozwala przy użyciu jednej funkcji założenie całej bazy danych z indeksami, tabelami i sekwencjami. Baza danych _sqlite_ jest wspierana w języku Python poprzez moduł `sqlite3` znajdujący się w bibliotece standardowej. Poniższy listing przedstawia prosty model danych wraz z przykładowym użyciem biblioteki `SqlAlchemy`.

In [96]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system

temp_db = mktemp(suffix='.sqlite')

print(f'Using {temp_db}')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}

# w Windows dodaje dodatkowo /
engine = create_engine(conn_uri_handler[system()])

Base = declarative_base(bind=engine)

class Product(Base):
    __tablename__ = 'products'
    id=Column(Integer, primary_key=True)
    title=Column('title', String(32))
    in_stock=Column('in_stock', Boolean)
    quantity=Column('quantity', Integer)


Base.metadata.create_all()

Session = sessionmaker(bind=engine)

@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func.__name__}')
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,) # dodanie parametru session do listy
                return func(*args, **kwargs)
        except Exception as e:
            print(f'Error found: {e}')

        return None

    return wrapper

@provide_session
def populatate(session):
    session.add(Product(id=1, title='Laptop Sony', in_stock=True, quantity=10))
    session.add(Product(id=2, title='Laptop Lenovo', in_stock=True, quantity=1))
    session.add(Product(id=3, title='Laptop Lenovo', in_stock=False, quantity=0))


@provide_session
def query(session):
    available_products = session.query(Product).filter(Product.in_stock == True).all()
    print('Available products')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')

    available_products = session.query(Product).filter(Product.in_stock == False).all()
    print('Not available products')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')


populatate()
query()

Using C:\Users\LUKASZ~1\AppData\Local\Temp\tmpkiy3tjw1.sqlite
Calling populatate
Calling query
Available products
1	Laptop Sony	10
2	Laptop Lenovo	1
Not available products
3	Laptop Lenovo	0


W zależności od tego co zostanie przekazane do funkcji `create_engine`, `SqlAlchemy` utworzy takie połączenie do bazy danych i skonwertuje typy języka Python na odpowiadające im typy w konkretnej bazie danych. Użyta tabela jest raczej trywialna, `SqlAlchemy` daje znacznie więcej możliwości np. tworzenia relacji (jeden do jeden, jeden do wielu, wiele do wielu), ograniczeń (_constraint_), wywoływania funkcji wbudowanych oraz tworzenia własnych definicji funkcji i typów, które zostaną przetransformowane do konkretnego dialektu (składnia danego języka zapytań obsługiwanego w bazie danych np. `t-sql` - _Microsoft SQL Server, 'pl/sql' - Oracle). W powyższym przykładzie, w funkcji `create_session` tworzona jest sesja do bazy danych. Użyty mechanizm pozwala na automatyczny `commit` i zwalnianie połączenia i przekazywanie go do puli otwartych połączeń. Dekorator `provide_session` dodaje parametr `session` do dekorowanej funkcji. Powyższy kod nie jest przeznaczony dla wielu wątków.

## Serializacja i deserializacja z modułem `pickle`

Moduł `pickle` umożliwia zapisywanie na dysk istniejących w pamięci instancji typów, które po ponownym uruchomieniu programu mogą być odczytywane z dysku. Nie każdy typ jest obsługiwany, ale kolekcje takie jak lista, słownik czy nazwana tupla są obsługiwane. Rozszerzeniem modułu `pickle` są moduły `dill` i `cpickle`, które obsługują większą liczbę typów.

In [76]:
from pickle import dump, load
from tempfile import mktemp
from os import unlink

temp_file = mktemp()

try:
    collection = ['A', 'B', 1, 2, True, False]
    print(f'Before: {collection}')
    with open(temp_file, 'wb') as f:
        dump(collection, f)
        f.flush()

    with open(temp_file, 'rb') as f:
        temp_col = load(f)
        print(f'After: {temp_col}')
finally:
    unlink(temp_file)


Before: ['A', 'B', 1, 2, True, False]
After: ['A', 'B', 1, 2, True, False]


## Pamięć podręczna i moduł _functools_

Twórcy języka Python wbudowali w moduły standardowe dekoratory, które umożliwiają tworzenie pamięci podręcznej (_cache_).
Dostępnych jest parę metod, niemniej jednak schemat użycia jest bardzo podobny.

In [8]:
from functools import lru_cache, cached_property


@lru_cache(maxsize=100)
def func_lru_cached(a: int, b: int):
    return a + b

glob_a, glob_b = 10,10

@cached_property
def sum_ab():
    return glob_a + glob_b

print(sum_ab)
print(func_lru_cached(glob_a, glob_b))

20


_lru_ oznacza _Least Recently Used_, co oznacza, że wywołania z parametrami rzadko używanymi będą usuwane z pamięci podręcznej w momencie kiedy tej pamięci podręcznej będzie brakować (parametr `maxsize`). Dekorator `cached_property` używany jest głównie do właściwości przechowywanych w klasach, które są przedmiotem kolejnego laboratorium.

## Zadania do wykonania

1. Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu `pickle`, `parquet` i `xlsx`.

2. Zbadaj przyspieszenie związane z zastosowaniem pamięci podręcznej na wybranych przykładzie funkcji rekurencyjnej (np. ciągu _Fibonacciego_.

3. Napisz program tworzący bazę danych z interfejsem konsolowym. Wymagane są następujące operacje, dodanie wiersza, usunięcie wiersza, zmiana pola wiersza, wyświetlenie opcji. Menu można zorganizować jako odczytywanie parametrów zwróconych przez funkcję `input` lub z pomocą komend (łatwiejszy sposób).

### Zadanie 1

In [8]:
import pickle
import time
from tempfile import mktemp
import pandas as pd
from os import unlink
from fastparquet import ParquetFile
from openpyxl import Workbook


my_dict = { 'A': list(range(0,100)) }
my_dict2 = { 'A': list(range(0,1000)) }
my_dict3 = { 'A': list(range(0,10000)) }


write_start_pickle_100 = time.time()
pickle.dump(my_dict, open("save.p", "wb"))
write_end_pickle_100 = time.time()
print('pickle, write 100 elements: ')
print(write_end_pickle_100 - write_start_pickle_100)

read_start_pickle_100 = time.time()
my_dict = pickle.load(open("save.p", "rb"))
read_end_pickle_100 = time.time()
print('pickle, read 100 elements: ')
print(read_end_pickle_100 - read_start_pickle_100)


write_start_pickle_1000 = time.time()
pickle.dump(my_dict2, open("save2.p", "wb"))
write_end_pickle_1000 = time.time()
print('pickle, write 1000 elements: ')
print(write_end_pickle_1000 - write_start_pickle_1000)

read_start_pickle_1000 = time.time()
my_dict2 = pickle.load(open("save2.p", "rb"))
read_end_pickle_1000 = time.time()
print('pickle, read 1000 elements: ')
print(read_end_pickle_1000 - read_start_pickle_1000)


write_start_pickle_10000 = time.time()
pickle.dump(my_dict3, open("save3.p", "wb"))
write_end_pickle_10000 = time.time()
print('pickle, write 10000 elements: ')
print(write_end_pickle_10000 - write_start_pickle_10000)

read_start_pickle_10000 = time.time()
my_dict3 = pickle.load(open("save3.p", "rb"))
read_end_pickle_10000 = time.time()
print('pickle, read 10000 elements: ')
print(read_end_pickle_10000 - read_start_pickle_10000)


# parquet


df = pd.DataFrame.from_dict(my_dict)
temp_file = mktemp()

try:
    write_start_parquet_100 = time.time()
    df.to_parquet(temp_file, compression='GZIP')
    pf = ParquetFile(temp_file)
    write_end_parquet_100 = time.time()
    print('parquet, write 100 elements: ')
    print(write_end_parquet_100 - write_start_parquet_100)
    read_start_parquet_100 = time.time()
    df = pd.read_parquet(temp_file)
    read_end_parquet_100 = time.time()
    print('parquet, read 100 elements: ')
    print(read_end_parquet_100 - read_start_parquet_100)
    
finally:
    unlink(temp_file)
    
    
df = pd.DataFrame.from_dict(my_dict2)

try:
    write_start_parquet_1000 = time.time()
    df.to_parquet(temp_file, compression='GZIP')
    pf = ParquetFile(temp_file)
    write_end_parquet_1000 = time.time()
    print('parquet, write 1000 elements: ')
    print(write_end_parquet_1000 - write_start_parquet_1000)
    read_start_parquet_1000 = time.time()
    df = pd.read_parquet(temp_file)
    read_end_parquet_1000 = time.time()
    print('parquet, read 1000 elements: ')
    print(read_end_parquet_1000 - read_start_parquet_1000)
    
finally:
    unlink(temp_file)
    
df = pd.DataFrame.from_dict(my_dict3)

try:
    write_start_parquet_10000 = time.time()
    df.to_parquet(temp_file, compression='GZIP')
    pf = ParquetFile(temp_file)
    write_end_parquet_10000 = time.time()
    print('parquet, write 10000 elements: ')
    print(write_end_parquet_10000 - write_start_parquet_10000)
    read_start_parquet_10000 = time.time()
    df = pd.read_parquet(temp_file)
    read_end_parquet_10000 = time.time()
    print('parquet, read 10000 elements: ')
    print(read_end_parquet_10000 - read_start_parquet_10000)
    
finally:
    unlink(temp_file)
    
    
# xlsx


df = pd.DataFrame()
df['Name'] = my_dict.keys()
df['Quantity'] = my_dict.values()

write_start_xlsx_100 = time.time()
df.to_excel('dict_test.xlsx')
write_end_xlsx_100 = time.time()
print('xlsx, write 100 elements: ')
print(write_end_xlsx_100 - write_start_xlsx_100)

read_start_xlsx_100 = time.time()
xlsx_read = pd.read_excel('dict_test.xlsx')
read_end_xlsx_100 = time.time()
print('xlsx, read 100 elements: ')
print(read_end_xlsx_100 - read_start_xlsx_100)



df['Name'] = my_dict2.keys()
df['Quantity'] = my_dict2.values()

write_start_xlsx_1000 = time.time()
df.to_excel('dict_test.xlsx')
write_end_xlsx_1000 = time.time()
print('xlsx, write 1000 elements: ')
print(write_end_xlsx_1000 - write_start_xlsx_1000)

read_start_xlsx_1000 = time.time()
xlsx_read = pd.read_excel('dict_test.xlsx')
read_end_xlsx_1000 = time.time()
print('xlsx, read 1000 elements: ')
print(read_end_xlsx_1000 - read_start_xlsx_1000)



df['Name'] = my_dict3.keys()
df['Quantity'] = my_dict3.values()

write_start_xlsx_10000 = time.time()
df.to_excel('dict_test.xlsx')
write_end_xlsx_10000 = time.time()
print('xlsx, write 10000 elements: ')
print(write_end_xlsx_10000 - write_start_xlsx_10000)

read_start_xlsx_10000 = time.time()
xlsx_read = pd.read_excel('dict_test.xlsx')
read_end_xlsx_10000 = time.time()
print('xlsx, read 10000 elements: ')
print(read_end_xlsx_10000 - read_start_xlsx_10000)

pickle, write 100 elements: 
0.0
pickle, read 100 elements: 
0.0009806156158447266
pickle, write 1000 elements: 
0.0
pickle, read 1000 elements: 
0.0019936561584472656
pickle, write 10000 elements: 
0.0009980201721191406
pickle, read 10000 elements: 
0.007051706314086914
parquet, write 100 elements: 
0.007642269134521484
parquet, read 100 elements: 
0.0019540786743164062
parquet, write 1000 elements: 
0.006974935531616211
parquet, read 1000 elements: 
0.0019521713256835938
parquet, write 10000 elements: 
0.012451648712158203
parquet, read 10000 elements: 
0.001995563507080078
xlsx, write 100 elements: 
0.008975028991699219
xlsx, read 100 elements: 
0.008977413177490234
xlsx, write 1000 elements: 
0.00997161865234375
xlsx, read 1000 elements: 
0.00897669792175293
xlsx, write 10000 elements: 
0.01691150665283203
xlsx, read 10000 elements: 
0.009972333908081055


### Zadanie 2

In [16]:
from functools import lru_cache, cached_property
import time

def fibonacci(n):
    if n < 0:
        print("Incorrect input")
    elif n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

n = 35
before_fib = time.time()
print(fibonacci(n))
after_fib = time.time()
print(f'Time to print {n}th Fibonacci sequence: ')
print(after_fib - before_fib)


@lru_cache(maxsize=100)
def fibonacci(n):
    if n < 0:
        print("Incorrect input")
    elif n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

n = 35
@cached_property
def fib():
    return fibonacci(n - 1) + fibonacci(n - 2)

before_fib_cache = time.time()
print(fibonacci(n))
after_fib_cache = time.time()
print(f'Time to print {n}th Fibonacci sequence: ')
print(after_fib_cache - before_fib_cache)


9227465
Time to print 35th Fibonacci sequence: 
5.165333271026611
9227465
Time to print 35th Fibonacci sequence: 
0.0


In [ ]:
### Zadanie 3

In [ ]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system

temp_db = mktemp(suffix='.sqlite')

print(f'Using {temp_db}')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}
engine = create_engine(conn_uri_handler[system()])

Base = declarative_base(bind=engine)

class Product(Base):
    __tablename__ = 'products'
    id=Column(Integer, primary_key=True)
    title=Column('title', String(32))
    quantity=Column('quantity', Integer)

Base.metadata.create_all()

Session = sessionmaker(bind=engine)

@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        #print(f'Calling {func.__name__}')
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,)
                return func(*args, **kwargs)
        except Exception as e:
            print(f'Error found: {e}')
        return None
    return wrapper

@provide_session
def add(session):
    print("Add product... ")
    id = input("Enter id: ")
    title = input("Enter title: ")
    quantity = input("Enter quantity: ")
    session.add(Product(id=id, title=title, quantity=quantity))
    session.commit()


@provide_session
def remove(session):
    id = input("Enter id of product you want to remove: ")
    session.query(Product).filter(Product.id == id).delete()
    session.commit()

@provide_session
def one_field(session):
    old_id = input("Enter id: ")
    field = input("Enter name of field you want to change: ")
    value = input("Enter new value: ")
    if field == "id":
        session.query(Product).filter(Product.id == old_id).update({Product.id:value}, synchronize_session = False)
    if field == "title":
        session.query(Product).filter(Product.id == old_id).update({Product.title:value}, synchronize_session = False)
    if field == "quantity":
        session.query(Product).filter(Product.id == old_id).update({Product.quantity:value}, synchronize_session = False)


@provide_session
def query(session):
    all_products = session.query(Product).all()
    print('All products: ')
    for product in all_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')

def run():
    n = 1
    while(n):
        display_main_menu()
        n = int(input("Select an option: "))
        if n == 1:
            add()
        if n == 2:
            remove()
        if n == 3:
            query()
        if n == 4:
            one_field()
        if n == 0:
            print("Program execution completed.")
            break


def display_main_menu():
    print('------- MENU -------')
    print(' 1. Add product')
    print(' 2. Delete product')
    print(' 3. View products')
    print(' 4. Change value of a specific field')

run()


Using C:\Users\Legion\AppData\Local\Temp\tmpl5wv32f1.sqlite
------- MENU -------
 1. Add product
 2. Delete product
 3. View products
 4. Change value of a specific field
